### Simulación de un día

In [16]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
import seaborn as sns
import os
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import datetime
from sklearn.cluster import KMeans
from geopy.distance import geodesic 
import warnings
from collections import defaultdict

#### Creación de un df combinado cronológico de salidas y llegadas (40 clusters)

In [24]:
df = pd.read_csv("df_with_clusters.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Filtrar los datos para usar solo los del día 26 de febrero de 2021 
df = df[(df['tsO'].dt.date == pd.to_datetime('2021-02-26').date())]
# Dividir el DataFrame en dos y hacer copias para evitar el SettingWithCopyWarning
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

# Añadir columna 'tipo' para indicar si es un cluster de origen o destino
df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

# Renombrar columnas para que tengan un nombre común ('ts' y 'cluster')
df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'cluster'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'cluster'}, inplace=True)

# Concatenar ambos DataFrames
df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)

# Ordenar por la columna temporal 'ts'
df_combinado.sort_values(by='ts', inplace=True)

# Restablecer el índice
df_combinado.reset_index(drop=True, inplace=True)

# Verificar el resultado
print(df_combinado.head()), len(df), len(df_combinado)


                   ts  cluster     tipo
0 2021-02-26 00:15:23        2   origen
1 2021-02-26 00:36:54       23   origen
2 2021-02-26 00:38:43       23  destino
3 2021-02-26 00:55:44       28   origen
4 2021-02-26 00:59:43        2  destino


(None, 1110, 2220)

In [26]:
# Filtrar los datos para quitar las llegadas en el día siguiente- 27 de febrero de 2021
df = df[(df['tsO'].dt.date == pd.to_datetime('2021-02-26').date())]

# Inicializar las estaciones
num_estaciones = 40
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Crear un DataFrame de eventos, con cada salida y llegada como eventos separados
df_salidas = df[['tsO', 'clusterO']].copy()
df_salidas['tipo'] = 'salida'
df_salidas.rename(columns={'tsO': 'timestamp', 'clusterO': 'nodo'}, inplace=True)

df_llegadas = df[['tsD', 'clusterD']].copy()
df_llegadas['tipo'] = 'llegada'
df_llegadas.rename(columns={'tsD': 'timestamp', 'clusterD': 'nodo'}, inplace=True)

# Concatenar salidas y llegadas y ordenar por timestamp
eventos = pd.concat([df_salidas, df_llegadas], ignore_index=True)
eventos.sort_values(by='timestamp', inplace=True)
eventos.reset_index(drop=True, inplace=True)

# Simulación de eventos
for _, evento in eventos.iterrows():
    nodo = evento['nodo']
    tipo = evento['tipo']

    if tipo == 'salida':
        # Si es una salida, disminuir el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] > 0:
            estaciones[nodo]['patinetes_disponibles'] -= 1
        else:
            # Si no hay patinetes disponibles, aumentar el número de patinetes iniciales, disponibles y slots
            estaciones[nodo]['patinetes_iniciales'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
            estaciones[nodo]['slots'] += 1

    elif tipo == 'llegada':
        # Si es una llegada, aumentar el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
            estaciones[nodo]['patinetes_disponibles'] += 1
        else:
            # Si todos los slots están ocupados, aumentar el número de slots
            estaciones[nodo]['slots'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
print("Simulación terminada")
# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots']
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones.csv', index=False)
print("Resultados guardados en dimensiones.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones.csv
Total de patinetes usados: 114


#### 79 estaciones

In [28]:
df = pd.read_csv("df_with_clusters2.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Filtrar los datos para usar solo los del día 26 de febrero de 2021 
df = df[(df['tsO'].dt.date == pd.to_datetime('2021-02-26').date())]
# Dividir el DataFrame en dos y hacer copias para evitar el SettingWithCopyWarning
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

# Añadir columna 'tipo' para indicar si es un cluster de origen o destino
df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

# Renombrar columnas para que tengan un nombre común ('ts' y 'cluster')
df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'cluster'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'cluster'}, inplace=True)

# Concatenar ambos DataFrames
df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)

# Ordenar por la columna temporal 'ts'
df_combinado.sort_values(by='ts', inplace=True)

# Restablecer el índice
df_combinado.reset_index(drop=True, inplace=True)

# Verificar el resultado
print(df_combinado.head()), len(df), len(df_combinado)


                   ts  cluster     tipo
0 2021-02-26 00:15:23       20   origen
1 2021-02-26 00:36:54        9   origen
2 2021-02-26 00:38:43        9  destino
3 2021-02-26 00:55:44       56   origen
4 2021-02-26 00:59:43       20  destino


(None, 1108, 2216)

In [30]:
# Filtrar los datos para quitar las llegadas en el día siguiente- 27 de febrero de 2021
df = df[(df['tsO'].dt.date == pd.to_datetime('2021-02-26').date())]

# Inicializar las estaciones
num_estaciones = 79
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Crear un DataFrame de eventos, con cada salida y llegada como eventos separados
df_salidas = df[['tsO', 'clusterO']].copy()
df_salidas['tipo'] = 'salida'
df_salidas.rename(columns={'tsO': 'timestamp', 'clusterO': 'nodo'}, inplace=True)

df_llegadas = df[['tsD', 'clusterD']].copy()
df_llegadas['tipo'] = 'llegada'
df_llegadas.rename(columns={'tsD': 'timestamp', 'clusterD': 'nodo'}, inplace=True)

# Concatenar salidas y llegadas y ordenar por timestamp
eventos = pd.concat([df_salidas, df_llegadas], ignore_index=True)
eventos.sort_values(by='timestamp', inplace=True)
eventos.reset_index(drop=True, inplace=True)

# Simulación de eventos
for _, evento in eventos.iterrows():
    nodo = evento['nodo']
    tipo = evento['tipo']

    if tipo == 'salida':
        # Si es una salida, disminuir el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] > 0:
            estaciones[nodo]['patinetes_disponibles'] -= 1
        else:
            # Si no hay patinetes disponibles, aumentar el número de patinetes iniciales, disponibles y slots
            estaciones[nodo]['patinetes_iniciales'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
            estaciones[nodo]['slots'] += 1

    elif tipo == 'llegada':
        # Si es una llegada, aumentar el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
            estaciones[nodo]['patinetes_disponibles'] += 1
        else:
            # Si todos los slots están ocupados, aumentar el número de slots
            estaciones[nodo]['slots'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
print("Simulación terminada")
# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots']
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones2.csv', index=False)
print("Resultados guardados en dimensiones2.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones2.csv
Total de patinetes usados: 175
